# NLP Exercises

We have five exercises in this section. The exercises are:
1. Build your own tokenizer, where you need to implement two functions to implement a tokenizer based on regular expression.
2. Get tags from Trump speech.
3. Get the nouns in the last 10 sentences from Trump's speech and find the nouns divided by sentencens. Use SpaCy.
4. Build your own Bag Of Words implementation using tokenizer created before.
5. Build a 5-gram model and clean up the results.

## Exercise 1. Build your own tokenizer

Build two different tokenizers:
- ``tokenize_sentence``: function tokenizing text into sentences,
- ``tokenize_word``: function tokenizing text into words.

In [24]:
from typing import List
import re
def tokenize_words(text: str) -> list:
    return re.findall(r'\b\w+\b', text)

def tokenize_sentence(text: str) -> list:
    return re.split('(?<=[.!?]) +(?=[:A-Z])', text)

text = "Here we go again. I was supposed to add this text later. \
Well, it's 10.p.m. here, and I'm actually having fun making this course. :o \
I hope you are getting along fine with this presentation, I really did try. \
And one last sentence, just so you can test you tokenizers better."

print("Tokenized sentences:")
print(tokenize_sentence(text))

print("Tokenized words:")
print(tokenize_words(text))

Tokenized sentences:
['Here we go again.', 'I was supposed to add this text later.', "Well, it's 10.p.m. here, and I'm actually having fun making this course.", ':o I hope you are getting along fine with this presentation, I really did try.', 'And one last sentence, just so you can test you tokenizers better.']
Tokenized words:
['Here', 'we', 'go', 'again', 'I', 'was', 'supposed', 'to', 'add', 'this', 'text', 'later', 'Well', 'it', 's', '10', 'p', 'm', 'here', 'and', 'I', 'm', 'actually', 'having', 'fun', 'making', 'this', 'course', 'o', 'I', 'hope', 'you', 'are', 'getting', 'along', 'fine', 'with', 'this', 'presentation', 'I', 'really', 'did', 'try', 'And', 'one', 'last', 'sentence', 'just', 'so', 'you', 'can', 'test', 'you', 'tokenizers', 'better']


## Exercise 2. Get tags from Trump speech using NLTK

You should use the ``trump.txt`` file, read it and find the tags for each word. Use NLTK for it.

In [25]:
import nltk
from nltk import word_tokenize
from nltk import pos_tag


nltk.download(quiet=True)

file = open("./datasets/trump.txt", "r",encoding="utf-8")
trump = file.read()
words = word_tokenize(trump)


tags = pos_tag(words)

for word, tag in tags[:20]:
    print(f"{word} -> {tag}")


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
Thank -> NNP
you -> PRP
very -> RB
much -> RB
. -> .
Mr. -> NNP
Speaker -> NNP
, -> ,
Mr. -> NNP
Vice -> NNP
President -> NNP
, -> ,
Members -> NNP
of -> IN
Congress -> NNP
, -> ,
the -> DT
First -> NNP
Lady -> NNP
of -> IN


## Exercise 3. Get the nouns in the last 10 sentences from Trump's speech and find the nouns divided by sentencens. Use SpaCy.

Please use Python list features to get the last 10 sentences and display nouns from it.

In [26]:
import spacy

nlp = spacy.load("en_core_web_sm")
file = open("./datasets/trump.txt", "r",encoding='utf-8')
trump = file.read()

doc = nlp(trump)
sentences = list(doc.sents)
last_10_sentences = sentences[-10:]

for i, sent in enumerate(last_10_sentences, 1):
    nouns = [token.text for token in sent if token.pos_ == "NOUN"]
    print(f"Sentence {i}:")
    print(f"Text: {sent.text.strip()}")
    print(f"Nouns: {nouns}\n")


Sentence 1:
Text: When we fulfill this vision, when we celebrate our 250 years of glorious freedom, we will look back on tonight as when this new chapter of American greatness began.
Nouns: ['vision', 'years', 'freedom', 'tonight', 'chapter', 'greatness']

Sentence 2:
Text: The time for small thinking is over.
Nouns: ['time', 'thinking']

Sentence 3:
Text: The time for trivial fights is behind us.
Nouns: ['time', 'fights']

Sentence 4:
Text: We just need the courage to share the dreams that fill our hearts, the bravery to express the hopes that stir our souls, and the confidence to turn those hopes and those dreams into action.
Nouns: ['courage', 'dreams', 'hearts', 'bravery', 'hopes', 'souls', 'confidence', 'hopes', 'dreams', 'action']

Sentence 5:
Text: From now on, America will be empowered by our aspirations, not burdened by our fears; inspired by the future, not bound by failures of the past; and guided by our vision, not blinded by our doubts.
Nouns: ['aspirations', 'fears', 'fut

## Exercise 4. Build your own Bag Of Words implementation using tokenizer created before 

You need to implement following methods:

- ``fit_transform`` - gets a list of strings and returns matrix with it's BoW representation
- ``get_features_names`` - returns list of words corresponding to columns in BoW

In [27]:
import numpy as np
import spacy

class BagOfWords:
    
    def __init__(self):
        self.__nlp = spacy.load("en_core_web_sm")
        self.__feature_names = []

    def fit_transform(self, corpus: list):
        vocab = {}
        tokenized_docs = []


        for doc in corpus:
            spacy_doc = self.__nlp(doc.lower())
            tokens = [token.text for token in spacy_doc if token.is_alpha and not token.is_stop]
            tokenized_docs.append(tokens)
            for token in tokens:
                if token not in vocab:
                    vocab[token] = len(vocab)

        self.__feature_names = [None] * len(vocab)
        for word, idx in vocab.items():
            self.__feature_names[idx] = word


        bow_matrix = np.zeros((len(corpus), len(vocab)), dtype=int)
        for i, tokens in enumerate(tokenized_docs):
            for token in tokens:
                bow_matrix[i, vocab[token]] += 1

        return bow_matrix

    def get_feature_names(self) -> list:
        return self.__feature_names


corpus = [
    'Bag Of Words is based on counting',
    'words occurences throughout multiple documents.',
    'This is the third document.',
    'As you can see most of the words occur only once.',
    'This gives us a pretty sparse matrix, see below. Really, see below',
]

vectorizer = BagOfWords()

X = vectorizer.fit_transform(corpus)
print(X)

print(vectorizer.get_feature_names())
print(len(vectorizer.get_feature_names()))


[[1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1]]
['bag', 'words', 'based', 'counting', 'occurences', 'multiple', 'documents', 'document', 'occur', 'gives', 'pretty', 'sparse', 'matrix']
13


## Exercise 5. Build a 5-gram model and clean up the results.

There are three tasks to do:
1. Use 5-gram model instead of 3.
2. Change to capital letter each first letter of a sentence.
3. Remove the whitespace between the last word in a sentence and . ! or ?.

Hint: for 2. and 3. implement a function called ``clean_generated()`` that takes the generated text and fix both issues at once. It could be easier to fix the text after it's generated rather then doing some changes in the while loop.

In [28]:
from nltk.book import *
import random
wall_street = text7.tokens

import re

tokens = wall_street

def cleanup():
    compiled_pattern = re.compile("^[a-zA-Z0-9.!?]")
    clean = list(filter(compiled_pattern.match,tokens))
    return clean
tokens = cleanup()

def build_ngrams():
    ngrams = []
    for i in range(len(tokens)-N+1):
        ngrams.append(tokens[i:i+N])
    return ngrams

def ngram_freqs(ngrams):
    counts = {}

    for ngram in ngrams:
        token_seq  = SEP.join(ngram[:-1])
        last_token = ngram[-1]

        if token_seq not in counts:
            counts[token_seq] = {}

        if last_token not in counts[token_seq]:
            counts[token_seq][last_token] = 0

        counts[token_seq][last_token] += 1;

    return counts

def next_word(text, N, counts):

    token_seq = SEP.join(text.split()[-(N-1):]);
    choices = counts[token_seq].items();

    total = sum(weight for choice, weight in choices)
    r = random.uniform(0, total)
    upto = 0
    for choice, weight in choices:
        upto += weight;
        if upto > r: return choice
    assert False # should not reach here

In [29]:
def clean_generated(text: str) -> str:
    text = re.sub(r"\s+([.?!])", r"\1", text)
    sentences = re.split(r'(?<=[.?!])\s+', text)
    sentences = [s.strip().capitalize() for s in sentences if s]
    return ' '.join(sentences)


N = 5
SEP = " "
sentence_count = 5
start_seq = random.choice(list(counts.keys()))

ngrams = build_ngrams()
counts = ngram_freqs(ngrams)


generated = start_seq.lower()
sentences = 0

while sentences < sentence_count:
    try:
        generated += SEP + next_word(generated, N, counts)
        if generated.endswith(('.', '!', '?')):
            sentences += 1
    except KeyError:

        new_start = random.choice(list(counts.keys()))
        print(f"Restarting from: '{new_start}'")
        generated += SEP + new_start


print("Before cleaning:\n")
print(generated)
print("\nAfter cleaning:\n")
print(clean_generated(generated))

Restarting from: 'embroiled in politics as'
Before cleaning:

banks . it the embroiled in politics as anti-abortion groups continued to oppose federal funding . The dispute has hampered the administration efforts to recruit prominent doctors 0 to fill prestigious posts at the helm of the NIH and the Centers for Disease Control . Several candidates have withdrawn their names from consideration after administration officials asked them for their views on abortion and fetal-tissue transplants . Antonio Novello whom Mr. Bush nominated to serve as surgeon general reportedly has assured the administration that she opposes abortion .

After cleaning:

Banks. It the embroiled in politics as anti-abortion groups continued to oppose federal funding. The dispute has hampered the administration efforts to recruit prominent doctors 0 to fill prestigious posts at the helm of the nih and the centers for disease control. Several candidates have withdrawn their names from consideration after administra